# MP 2: Geometry and Contact

**Due date**: Feb 11, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. To submit your assignment, perform the following:

1. Double-check that your notebook runs without error from a clean "Restart and Clear Output".
2. Download your completed notebook.  To avoid sending unnecessarily large notebooks, choose "Restart and Clear Output" before downloading. 
3. Submit your notebook (and any other new files you created) on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).

In [ ]:
#Imports

#If you have wurlitzer installed, this will help you catch printouts from Klamp't
#Note: doesn't work on Windows
#%load_ext wurlitzer

import time
from klampt import *
from klampt import vis
from klampt.math import vectorops,so3,se3
from klampt.model.trajectory import Trajectory
from klampt.io import numpy_convert
import numpy as np
import math
import random
vis.init('IPython')
pass

## Problem 1: Normal Calculation

Let us examine the problem of calculating geometry normals.  The code below will load triangle mesh files, which are produced by CAD programs and mature 3D scanning pipelines, and will also generate point clouds, which are produced by raw 3D scanning from laser sensors and RGB-D cameras.

A. In the first marked cell, implement a function that calculates the normals of each triangle in the mesh. *Recall that a normal is a unit vector pointing to the outside of the geometry.*  Use the cell afterwards to debug your estimates.  The results will appear in the visualization window under the cell below.

B. In the second marked cell, use the [klampt.model.geometry.point_cloud_normals()](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/_modules/klampt/model/geometry.html#point_cloud_normals) function to produce estimates of normals for a point cloud.  There are several parameters to this function that affect the quality of the normal estimates for varying resolutions and noise characteristics.  Tune these parameters to give "reasonable" estimates at least when noise is slow.

   Implement the `normal_rmse` function to comptue the root-mean-squared-error (RMSE) between the ground truth normals and the ones calculated by your function.  For two sets of points $(x_1,...,x_N)$ and $(y_1,...,y_N)$, RMSE is given by:
    
   $$ \sqrt{1/N \sum_{i=1}^N \|x_i-y_i\|^2} $$

   Describe trends in the estimation quality as a function of the `estimation_radius` and `estimation_knn` parameters, and well as the resolutions and noise characteristics.  Vary the parameters to at least 3 values to better understand these trends.  Place your answers in the space designated for written answers.

In [ ]:
world = WorldModel()
obj = world.makeRigidObject("object1")
#obj.geometry().loadFile("../data/objects/ycb-select/002_master_chef_can/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/003_cracker_box/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/011_banana/nontextured.ply"); 
#obj.geometry().loadFile("../data/objects/ycb-select/048_hammer/nontextured.ply")
obj.geometry().loadFile("../data/objects/cube.off"); obj.geometry().scale(0.2)
#weird bug in Qhull -- cylinder can't be converted to ConvexHull
#obj.geometry().loadFile("../data/objects/cylinder.off")

#this will perform a reasonable center of mass / inertia estimate
m = obj.getMass()
m.estimate(obj.geometry(),mass=0.454,surfaceFraction=0.2)
obj.setMass(m)

#make the object transparent yellow
obj.appearance().setColor(0.8,0.8,0.2,0.5)
#draw center of mass
vis.createWindow()
vis.add("world",world)
vis.add("COM",m.getCom(),color=(1,0,0,1),size=0.01)
vis.show()

In [ ]:
########################## Problem 1.A code goes in this cell ##############################

def get_triangle_normals(verts,tris):
    """
    Returns a list or numpy array of (outward) triangle normals for the
    triangle mesh defined by vertices verts and triangles tris.
    
    Args:
        verts: a Numpy array with shape (numPoints,3)
        tris: a Numpy int array with shape (numTris,3)
    """
    #TODO: put your code here
    normals = np.zeros(tris.shape)
    return normals

###########################################################################################

In [ ]:
#this cell tests your code -- you should see points sticking out of your model like a "hedgehog"
xform, (verts, tris) = numpy_convert.to_numpy(obj.geometry())
trinormals = get_triangle_normals(verts,tris)
    
nlen = 0.02
skip = 1
if len(tris) > 200:
    skip = len(tris)//50
for i in range(0,len(tris),skip):
    a,b,c = tris[i]
    center = (verts[a] + verts[b] + verts[c])/3
    path = [center.tolist(),vectorops.madd(center,trinormals[i],nlen)]
    vis.add("normal{}".format(i),path,color=(1,1,0,1))

In [ ]:
########################## Problem 1.B code goes in this cell ##############################

from klampt.model.geometry import point_cloud_normals

def get_point_cloud_normals(pc):
    """Returns a numPoints() x 3 array of normals, one for each point.
    
    pc is a klampt.PointCloud object.  
    """
    #Note: if you want the points of the point cloud as a Numpy array, you can use
    #pts = numpy_convert.to_numpy(pc)
    normals = point_cloud_normals(pc,add=False)
    return normals

def normal_rmse(normals1,normals2):
    """Given two numPoints() x 3 arrays of normals, computes the
    root mean squared error (RMSE) between the normals.
    """
    return 0

###########################################################################################

In [ ]:
#this cell tests your code -- you should see three new point cloud geometries, and normals
#sticking out of those models like a "hedgehog"

resolutions = [0.05,0.02,0.01,
               0.05,0.02,0.01]
noises = [0,0,0,
         0.01,0.01,0.01]
translations = [[0.3,0,0],[0.6,0,0],[0.9,0,0],
                [0.3,0.3,0],[0.6,0.3,0],[0.9,0.3,0]]

tests = list(range(len(resolutions)))

pcs = []
for j in tests:
    res,noise,translation = resolutions[j],noises[j],translations[j]
    
    pc = obj.geometry().convert('PointCloud',res)
    (xform,A) = numpy_convert.to_numpy(pc)
    ground_truth_normals = A[:,3:6]
    A = A[:,:3]
    if noise != 0:
        A[:,:] += np.random.normal(scale=noise,size=(A.shape[0],3))
    pc.setPointCloud(numpy_convert.from_numpy(A,'PointCloud'))
    pc.translate(translation)
    pcs.append(pc)

    pc_normals = get_point_cloud_normals(pc)

    skip = 1
    nlen = 0.02
    if len(pc_normals) > 200:
        skip = len(pc_normals)//50
    pc_data = pc.getPointCloud()
    vis.add('pc{}'.format(j),pc,color=(1,0,1,0.5))
    for i in range(0,len(pc_normals),skip):
        center = pc_data.getPoint(i)
        path = [center,vectorops.madd(center,pc_normals[i],nlen)]
        vis.add("pc{}_normal{}".format(j,i),path,color=(1,1,0,1))

### Problem 1.B Written Answers

**TODO: Your answers to the written questions should go in this cell**

## Problem 2: Planning antipodal grasps

Problem 2 asks to find antipodal grasps for a given object geometry, that is, line segments that a gripper could plausibly use for closing onto the geometry.  Here we will use the `AntipodalGrasp` data structure to represent these grasps, as well as the constants at the top of the next cell to represent prior knowledge about the gripper's geometry.

The key defining parameters of an antipodal grasp include the center of the line segment and its axis. From these parameters, you should be able to fill in its contact points and the grasp width using some geometric computations.  Specifically, the [Geometry3D.rayCast_ext](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html#klampt.Geometry3D.rayCast_ext) method used in `ray_cast_to_contact_point` will be helpful for you to retrieve the normal and position of a contact point.

In [ ]:
########################## Problem 2 code goes in this cell ##############################

from klampt.model.contact import ContactPoint

max_grasp_width = 0.14
max_grasp_depth = 0.12
finger_radius = 0.01
grasp_width_uncertainty = 0.02
gravity = (0,0,-9.8)
object_normals = None

class AntipodalGrasp:
    """A structure containing information about antipodal grasps.
    
    Attributes:
        center (3-vector): the center of the fingers (object coordinates).
        axis (3-vector): the direction of the line through the
            fingers (object coordinates).
        approach (3-vector, optional): the direction that the fingers
            should move forward to acquire the grasp.
        finger_width (float, optional): the width that the gripper should
            open between the fingers.
        contact1 (ContactPoint, optional): a point of contact on the
            object.
        contact2 (ContactPoint, optional): another point of contact on the
            object.
    """
    def __init__(self,center,axis):
        self.center = center
        self.axis = axis
        self.approach = None
        self.finger_width = None
        self.contact1 = None
        self.contact2 = None

    def add_to_vis(self,name,color=(1,0,0,1)):
        if self.finger_width == None:
            w = 0.05
        else:
            w = self.finger_width*0.5+finger_radius
        a = vectorops.madd(self.center,self.axis,w)
        b = vectorops.madd(self.center,self.axis,-w)
        vis.add(name,[a,b],color=color)
        if self.approach is not None:
            vis.add(name+"_approach",[self.center,vectorops.madd(self.center,self.approach,0.05)],color=(1,0.5,0,1))

def grasp_from_contacts(contact1,contact2):
    """Helper: if you have two contacts, this returns an AntipodalGrasp"""
    d = vectorops.unit(vectorops.sub(contact2.x,contact1.x))
    grasp = AntipodalGrasp(vectorops.interpolate(contact1.x,contact2.x,0.5),d)
    grasp.finger_width = vectorops.distance(contact1.x,contact2.x)
    grasp.contact1 = contact1
    grasp.contact2 = contact2
    return grasp

def ray_cast_to_contact_point(obj,source,direction):
    """Produces a ContactPoint for the point hit by the ray:
        x(t) = source + t*direction
    
    Assumes object_normals is set up to match the object.
    """
    hit_tri,pt = obj.geometry().rayCast_ext(source,direction)
    if hit_tri < 0:
        return None
    return ContactPoint(pt,object_normals[hit_tri],1)

def get_contacts(grasp,rigid_object):
    """TODO: you may want to fill this out to generate
    grasp.contact1,grasp.contact2 and grasp.finger_width...
    """
    #perhaps doing some ray casting would help here?
    return
    
def within_fingers(grasp,rigid_object):
    """Returns 1 if the object fits within max_grasp_width-grasp_width_uncertainty
    units along the specified grasp axis, and 0 if it has width greater than 
    max_grasp width, and otherwise decaying linearly from 1 to 0.
    
    You may want to create a helper function to fill in grasp.contact1 and
    grasp.contact2 if they aren't already there.
    """
    #TODO: your code here
    return 1

def centering_alignment(grasp,rigid_object):
    """Returns the absolute error between the grasp center and the midpoint 
    between the points touched on the object.  If c is the center and
    m is the midpoint between the contact points p1 and p2, then this
    returns ||c-m||.
    
    You may want to create a helper function to fill in grasp.contact1 and
    grasp.contact2 if they aren't already there.
    """
    #TODO: your code here
    return 0
            
def normal_alignment(grasp,rigid_object):
    """Returns the total angular error between the grasp axis and the normals
    of the object.  Specifically, returns angle(axis,n1) + angle(axis,n2) in
    radians, with n1 and n2 being the normals of the contact points.  (Choose
    the sign of axis that minimizes the angle.)
    
    You may want to create a helper function to fill in grasp.contact1 and
    grasp.contact2 if they aren't already there.
    """
    #TODO: your code here
    return 0

def off_axis_torque(grasp,rigid_object):
    """Returns the magnitude of the torque imposed by gravity on the object
    for the given AntipodalGrasp.  (Don't forget to include the object's mass
    in your calculation).
    """
    #TODO: your code here
    return 0

def finger_clearance(grasp,rigid_object):
    """Returns how much clearance the fingers have about the grasp.
    
    Imagine a point finger_radius away from the contact points, this
    should return min(d/finger_radius,1) where d is the minimum
    distance between the point and surrounding obstacles.
    """
    #TODO: your code here
    return 1

def antipodal_grasp_score(grasp,rigid_object):
    """Returns a score indicating a quality of the grasp. Lower is better.
    """
    within = within_fingers(grasp,rigid_object)
    center = centering_alignment(grasp,rigid_object)
    normal = normal_alignment(grasp,rigid_object)
    off_axis = off_axis_torque(grasp,rigid_object)
    clearance = finger_clearance(grasp,rigid_object)
    #print(within,center,normal,off_axis,clearance)
    if within == 0: return float('inf')
    #TODO: weight between these components
    return (1.0-within) + center + normal + off_axis + (1.0-clearance)

def antipodal_grasp_sample_volume(rigid_object,k,N):
    """Samples the top k high quality antipodal grasps for the
    object out of N sampled centers and axes within the object
    
    Returns a list of up to k (grasp,score) pairs, sorted in order
    of increasing score.
    """
    #this is a bounding box for the object
    bmin,bmax = rigid_object.geometry().getBBTight()
    scores = []
    for i in range(N):
        #TODO do something to sample a grasp. Then, append (score,grasp) to scores
        pass
    scores = sorted(scores,key = lambda x:x[0])
    return [(x[1],x[0]) for x in scores[:min(k,len(scores))]]

def sample_trimesh(verts,tris,N):
    """Samples N points and normals from a triangle mesh"""
    global object_normals
    sample = [random.randint(0,len(verts)-1) for i in range(N)]
    pts = []
    normals = []
    for elem in sample:
        a,b,c = tris[elem]
        va = verts[a]
        vb = verts[b]
        vc = verts[c]
        u = random.random()
        v = random.random()
        if u+v > 1:
            v2 = (1-u)
            u2 = (1-v)
            u,v = u2,v2
        pt = va*(1-u-v) + vb*u + vc*v
        n = object_normals[elem]
        pts.append(pt)
        normals.append(n)
    return pts,normals

def antipodal_grasp_sample_surface(rigid_object,k,N):
    """Samples the top k high quality antipodal grasps for the
    object out of N sampled pairs of points on the surface of
    the object.
    
    Returns a list of up to k (grasp,score) pairs, sorted in order
    of increasing score.
    """
    verts,tris = numpy_convert.to_numpy(rigid_object.geometry().getTriangleMesh())
    scores = []
    for i in range(N):
        #TODO do something to sample a grasp. Then, append (score,grasp) to scores
        pass
    scores = sorted(scores,key = lambda x:x[0])
    return [(x[1],x[0]) for x in scores[:min(k,len(scores))]]


###########################################################################################

A. In the following cell you will be given some good grasps and bad grasps.  Fill in the components of the `antipodal_grasp_score` function to generate a scoring function for each antipodal grasp.  Your scoring function should implement at least 3 of the `within_fingers`, `centering_alignment`, `normal_alignment`, `off_axis_torque`, and `finger_clearance` features as described in their documentation.  The final score should weight between them.  Grasps that are obviously good should show up as green, and grasps that are obviously bad should show up as red.

In [ ]:
#Problem 2.A. Basic test of the quality of one obviously good grasp and two obviously bad ones.
#Feel free to debug your code by testing new grasps here
vis.createWindow()
vis.add("world",world)

object_normals = trinormals
grasp1 = AntipodalGrasp([0,0,0.02],[1,0,0])
grasp2 = AntipodalGrasp([0,0.2,0.02],[1,0,0])
grasp3 = AntipodalGrasp([0,0,0.02],[0,1,0])
grasps = [(g,antipodal_grasp_score(g,obj)) for g in [grasp1,grasp2,grasp3]]
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
vis.show()

B. Define a volume-centric grasp selection strategy in `antipodal_grasp_sample_volume` that samples grasps by sampling a random center point in the middle of the object's volume, and a random direction for the axis.  From these N grasps, it should score and choose the top k grasps. The third cell below will show the results of your grasp sampler on whatever object was loaded in the initial cell at the top of this workbook.  By changing the object loaded (try the banana and hammer!), you should see different behavior.

In [ ]:
#Problem 2.B. Run your volume-based grasp sampler.
vis.createWindow()
#uncomment this if you want to try your hand at point cloud grasp generation
"""
if world.numRigidObjects() == 1:
    obj_pc = world.makeRigidObject("object1_pc")
    obj_pc.geometry().set(obj.geometry().convert('PointCloud',0.02))
    translation = (0.3,0,0)
    m = obj.getMass()
    m.setCom(vectorops.add(m.getCom(),translation))
    obj_pc.setMass(m)
    obj_pc.geometry().translate(translation)
    obj_pc.appearance().setColor(1,0,1,0.5)
    pc_normals = get_point_cloud_normals(obj_pc.geometry().getPointCloud())
    obj = world.rigidObject(0)
"""
vis.add("world",world)

object_normals = trinormals
t0 = time.time()
grasps = antipodal_grasp_sample_volume(obj,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 

#uncomment this if you want to try your hand at point cloud grasp generation
"""
object_normals = pc_normals
t0 = time.time()
grasps = antipodal_grasp_sample_volume(obj_pc,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "pc_grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
"""
vis.show()

C. Define a surface-centric grasp selection strategy in `antipodal_grasp_sample_surface` that samples grasps by sampling points on the object at random.  

In [ ]:
#Problem 2.C. Run your surface-based grasp sampler.
vis.createWindow()
vis.add("world",world)

object_normals = trinormals
t0 = time.time()
grasps = antipodal_grasp_sample_surface(obj,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
    
#uncomment this if you want to try your hand at point cloud grasp generation
"""
object_normals = pc_normals
t0 = time.time()
grasps = antipodal_grasp_sample_surface(obj_pc,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "pc_grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
"""
vis.show()

## Problem 3: Finding stable orientations

Finally, in Problem 3 you will generate a small world with several objects on a flat table.  The `stable_faces` function provided will calculate a set of faces that the object can use to stably rest on a flat surface. (It does use your `get_triangle_normals` function, so you will need to complete Problem 1.A first.)

In the `sample_arrangement()` function, sample k copies of the object so that they rest on the table with orientations chosen randomly from the stable faces.  Once you create a `RigidObjectModel` using `WorldModel.makeRigidObject()` and copy its geometry and mass, you should set its current transform using `RigidObjectModel.setTransform()`.  (Do not modify the geometry itself using `object.geometry().transform()`)

In [ ]:
from collections import deque
from scipy.spatial import ConvexHull

def stable_faces(obj,com=None,stability_tol=0.0,merge_tol=0.0):
    """
    Returns a list of support polygons on the object that are
    likely to be stable on a planar surface.
    
    Args:
        obj (RigidObjectModel or Geometry3D): the object.
        com (3-list, optional): sets the local center of mass. If
            not given, the default RigidObjectModel's COM will be used,
            or (0,0,0) will be used for a Geometry3D.
        stability_tol (float,optional): if > 0, then only faces that
            are stable with all perturbed "up" directions (dx,dy,1) with
            ||(dx,dy)||<= normal_tol will be outputted (robust stability). 
            If < 0, then all faces that are stable from some "up" direction
            (dx,dy,1) with ||(dx,dy)||<= |normal_tol| will be outputted
            (non-robust stability)
        merge_tol (float, optional): if > 0, then adjacent faces with
            normals whose angles are within this tolerance (in rads) will
            be merged together.
    
    Returns:
        list of list of 3-vectors: The set of all polygons that could
        form stable sides. Each polygon is convex and listed in
        counterclockwise order (i.e., the outward normal can be obtained
        via:
        
            (poly[2]-poly[0]) x (poly[1]-poly[0])
        
    """
    if isinstance(obj,RigidObjectModel):
        geom = obj.geometry()
        if com is None:
            com = obj.getMass().getCom()
    else:
        geom = obj
        if com is None:
            com = (0,0,0)
    assert len(com) == 3,"Need to provide a 3D COM"
    ch_trimesh = geom.convert('ConvexHull').convert('TriangleMesh')
    xform, (verts, tris) = numpy_convert.to_numpy(ch_trimesh)
    trinormals = get_triangle_normals(verts,tris)
    
    edges = dict()
    tri_neighbors = np.full(tris.shape,-1,dtype=np.int32)
    for ti,tri in enumerate(tris):
        for ei,e in enumerate([(tri[0],tri[1]),(tri[1],tri[2]),(tri[2],tri[0])]):
            if (e[1],e[0]) in edges:
                tn,tne = edges[(e[1],e[0])]
                if tri_neighbors[tn][tne] >= 0:
                    print("Warning, triangle",ti,"neighbors two triangles on edge",tne,"?")
                tri_neighbors[ti][ei] = tn
                tri_neighbors[tn][tne] = ti
            else:
                edges[e] = ti,ei
    num_empty_edges = 0
    for ti,tri in enumerate(tris):
        for e in range(3):
            if tri_neighbors[tn][e] < 0:
                num_empty_edges += 1
    if num_empty_edges > 0:
        print("Info: boundary of mesh has",num_empty_edges,"edges")
    visited = [False]*len(tris)
    cohesive_faces = dict()
    for ti,tri in enumerate(tris):
        if visited[ti]:
            continue
        face = [ti]
        visited[ti] = True
        myvisit = set()
        myvisit.add(ti)
        q = deque()
        q.append(ti)
        while q:
            tvisit = q.popleft()
            for tn in tri_neighbors[tvisit]:
                if tn >= 0 and tn not in myvisit:
                    if math.acos(trinormals[ti].dot(trinormals[tn])) <= merge_tol:
                        face.append(tn)
                        myvisit.add(tn)
                        q.append(tn)
        for t in myvisit:
            visited[t] = True
        cohesive_faces[ti] = face
    output = []
    for t,face in cohesive_faces.items():
        n = trinormals[t]
        R = so3.canonical(n)
        if len(face) > 1:
            #project face onto the canonical basis
            faceverts = set()
            for t in face:
                faceverts.add(tris[t][0])
                faceverts.add(tris[t][1])
                faceverts.add(tris[t][2])
            faceverts = list(faceverts)
            xypts = [so3.apply(so3.inv(R),verts[v])[1:3] for v in faceverts]
            try:
                ch = ConvexHull(xypts)
                face = [faceverts[v] for v in ch.vertices]
            except Exception:
                print("Error computing convex hull of",xypts)
                print("Vertex indices",faceverts)
                print("Vertices",[verts[v] for v in faceverts])
        else:
            face = tris[face[0]]
        comproj = np.array(so3.apply(so3.inv(R),com)[1:3])
        
        stable = True
        for vi in range(len(face)):
            vn = (vi+1)%len(face)
            a,b = face[vi],face[vn]
            pa = np.array(so3.apply(so3.inv(R),verts[a])[1:3])
            pb = np.array(so3.apply(so3.inv(R),verts[b])[1:3])
            #check distance from com
            elen = np.linalg.norm(pb-pa)
            if elen == 0:
                continue
            sign = np.cross(pb - pa,comproj-pa)/elen
            if sign < stability_tol:
                stable = False
                break
        if stable:
            output.append([verts[i] for i in face])
    return output
        

In [ ]:
#debugging the stable faces
sides = stable_faces(obj,stability_tol=-0.01,merge_tol=0.05)
vis.createWindow()
for i,f in enumerate(sides):
    gf = GeometricPrimitive()
    gf.setPolygon(np.stack(f).flatten())
    color = (0.5+0.5*random.random(),0.5+0.5*random.random(),0.5+0.5*random.random(),1)
    vis.add("face{}".format(i),Geometry3D(gf),color=color)
vis.show()

In [ ]:
from klampt.model.create import primitives
table_world = WorldModel()
table_width = 0.5
table_depth = 0.5
table_height = 0.3
table_surf = primitives.box(table_width,table_depth,0.05,center=(0,0,table_height-0.025),world=table_world,mass=float('inf'))
table_legs = []
table_legs.append(primitives.box(0.05,0.05,table_height,center=(-table_width*0.5+0.05,-table_depth*0.5+0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(table_width*0.5-0.05,-table_depth*0.5+0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(-table_width*0.5+0.05,table_depth*0.5-0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(table_width*0.5-0.05,table_depth*0.5-0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_surf.appearance().setColor(0.4,0.3,0.2)
for l in table_legs:
    l.appearance().setColor(0.4,0.3,0.2)

def make_object(clone_obj,world):
    obj = world.makeRigidObject("object"+str(world.numRigidObjects()))
    obj.geometry().set(clone_obj.geometry())
    obj.appearance().set(clone_obj.appearance())
    obj.setMass(clone_obj.getMass())
    return obj

#copy from previous cell's computation
obj_stable_faces = stable_faces(obj,stability_tol=-0.001,merge_tol=0.05)

########################## Problem 3 code goes in this cell ##############################

def sample_arrangement(k):
    for i in range(k):
        objsample = make_object(obj,table_world)
        R = so3.identity()
        t = [0,0,0]
        #TODO: do something to sample a rotation and translation
        objsample.setTransform(R,t)

##########################################################################################

sample_arrangement(5)

vis.createWindow()
vis.add("world",table_world)
vis.show()